# Vector stores and retrievers
Adapted from [Vector stores and retrievers](https://python.langchain.com/v0.2/docs/tutorials/retrievers/)

This tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation, or RAG (see our RAG tutorial [here](https://python.langchain.com/v0.2/docs/tutorials/rag/)).

## Concepts
This guide focuses on retrieval of text data. We will cover the following concepts:

* Documents
* Vector stores
* Retrievers

### Installation
This tutorial requires the `langchain`, `langchain-chroma`, and `langchain-openai` packages:

```bash
pip install langchain langchain-chroma langchain-openai
```

## Credentials and Langchain
Load credentials from a .env file and the [python-dotenv package](https://pypi.org/project/python-dotenv/)

```toml
LANGCHAIN_API_KEY="<KEY>"
OPENAI_API_KEY="<KEY>"
```

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with [LangSmith](https://smith.langchain.com/).

After you sign up at the link above, make sure to set your environment variables to start logging traces:
```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

In [3]:
import os
from dotenv import load_dotenv

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
assert os.environ["LANGCHAIN_API_KEY"]
assert os.environ["OPENAI_API_KEY"]

## Documents
LangChain implements a [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

* `page_content`: a string representing the content;
* `metadata`: a dict containing arbitrary metadata.
The `metadata` attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual `Document` object often represents a chunk of a larger document.

Let's generate some sample documents:

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

* API Reference: [Document](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#langsmith:~:text=API%20Reference%3A,three%20distinct%20%22sources%22.)

Here we've generated five documents, containing metadata indicating three distinct "sources".

## Vector stores
Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can embed it as a vector of the same dimension and use vector similarity metrics to identify related data in the store.

LangChain [VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) objects contain methods for adding text and Document objects to the store, and querying them using various similarity metrics. They are often initialized with [embedding models](https://python.langchain.com/v0.2/docs/how_to/embed_text/), which determine how text data is translated to numeric vectors.

LangChain includes a [suite of integrations](https://python.langchain.com/v0.2/docs/integrations/vectorstores/) with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as Postgres) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads. Here we will demonstrate usage of LangChain VectorStores using Chroma, which includes an in-memory implementation.

To instantiate a vector store, we often need to provide an embedding model to specify how text should be converted into a numeric vector. Here we will use [OpenAI embeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/).

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Create a vectorstore and and add documents
vectorstore = Chroma.from_documents(
    documents,                      # List of documents created earlier
    embedding=OpenAIEmbeddings(),   # Use the OpenAI embeddings
)

* API Reference: [OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)

Calling `.from_documents` here will add the documents to the vector store. VectorStore implements methods for adding documents that can also be called after the object is instantiated. Most implementations will allow you to connect to an existing vector store-- e.g., by providing a client, index name, or other information. See the documentation for a specific integration for more detail.

Once we've instantiated a `VectorStore` that contains documents, we can query it. VectorStore includes methods for querying:

* Synchronously and asynchronously;
* By string query and by vector;
* With and without returning similarity scores;
* By similarity and [maximum marginal relevance](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search) (to balance similarity with query to diversity in retrieved results).

The methods will generally include a list of Document objects in their outputs.

### Examples
Return documents based on similarity to a string query:



In [7]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

Async query:

In [8]:
await vectorstore.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

Return scores:

In [9]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.37532690167427063),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.4833090305328369),
 (Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  0.4958883225917816),
 (Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  0.4974174499511719)]

Return documents based on similarity to a embedded query:

In [16]:
embedding = OpenAIEmbeddings().embed_query("cat")

# An embedding is a list or matrix of floating point numbers readily readable by an LLM
print(f"First 5 embeddings: {embedding[0:5]}")

vectorstore.similarity_search_by_vector(embedding)

First 5 embeddings: [-0.007064840399927003, -0.017335813760570657, -0.009703516178850385, -0.03069942864557231, -0.012505334995320574]


[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

Learn more:

* [API reference](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html)
* [How-to guide](https://python.langchain.com/v0.2/docs/how_to/vectorstores/)
* [Integration-specific docs](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

## Retrievers
LangChain `VectorStore` objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous `invoke` and `batch` operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing `Retriever`. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the `similarity_search` method:

In [19]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# Create a Runnable by passing the .similarity_search method of vectorstore in
# Pick the top result, by passing in param k=1
retriever = RunnableLambda(vectorstore.similarity_search) \
            .bind(k=1)  # select top result

# Send in 2 queries, first cat the shark
retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

* API Reference: [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) | [RunnableLambda](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html)

Vectorstores implement an `as_retriever` method that will generate a Retriever, specifically a [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html). These retrievers include specific `search_type` and `search_kwargs` attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [20]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

`VectorStoreRetriever` supports search types of `"similarity"` (default), `"mmr"` (maximum marginal relevance, described above), and `"similarity_score_threshold"`. We can use the latter to threshold documents output by the retriever by similarity score.

Retrievers can easily be incorporated into more complex applications, such as retrieval-augmented generation (RAG) applications that combine a given question with retrieved context into a prompt for a LLM. Below we show a minimal example.

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

# The context is provider by the retriever object which returns the top result from the vectorstore by similarity score
# Runnable to passthrough inputs unchanged or with additional keys.
# Pass context into prompt 
# Send prompt to LLM, which is the OpenAI model
rag_chain = {"context": retriever, \
             "question": RunnablePassthrough()} \
            | prompt \
            | llm

API Reference: [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) | [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)

In [24]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

Cats are independent pets that often enjoy their own space.


## Learn more:
Retrieval strategies can be rich and complex. For example:

* We can [infer hard rules and filters](https://python.langchain.com/v0.2/docs/how_to/self_query/) from a query (e.g., "using documents published after 2020");
* We can [return documents that are linked](https://python.langchain.com/v0.2/docs/how_to/parent_document_retriever/) to the retrieved context in some way (e.g., via some document taxonomy);
* We can generate [multiple embeddings](https://python.langchain.com/v0.2/docs/how_to/multi_vector/) for each unit of context;
* We can [ensemble results](https://python.langchain.com/v0.2/docs/how_to/ensemble_retriever/) from multiple retrievers;
* We can assign weights to documents, e.g., to [weigh recent documents higher](https://python.langchain.com/v0.2/docs/how_to/time_weighted_vectorstore/).

The [retrievers section](https://python.langchain.com/v0.2/docs/how_to/#retrievers) of the how-to guides covers these and other built-in retrieval strategies.

It is also straightforward to extend the [BaseRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html) class in order to implement custom retrievers. See our how-to guide [here](https://python.langchain.com/v0.2/docs/how_to/custom_retriever/).